In [ ]:
# https://www.kaggle.com/ekrembayar/an-introduction-to-football-analytics

In [ ]:
!pip install pyjsparser
!pip install js2xml

  Created wheel for pyjsparser: filename=pyjsparser-2.7.1-py3-none-any.whl size=26000 sha256=3a6d691f4ef4fb4aa60912670f93c660f259d95663aabbb7ae3fc2a60a5aa625
  Stored in directory: /root/.cache/pip/wheels/2a/80/ac/dcd2bdbd03dd2b7b7e2bf3e5afbda6a1ab7935bbce314969da
Successfully built pyjsparser
     |████████████████████████████████| 291 kB 25.0 MB/s 
     |████████████████████████████████| 49 kB 7.2 MB/s 


In [ ]:
# Base
import pandas as pd
import numpy as np

# Visualization
import plotly.express as ex

# Web Scraping
import requests
from bs4 import BeautifulSoup
# !pip install pyjsparser
# from pyjsparser import parse
import pyjsparser

# Time Sleep
import sys
import time
from datetime import datetime
from termcolor import colored

# GC
import gc

# Itertools
import itertools

# Grafikten Data Çekmek için
import re
import js2xml
from itertools import repeat    
from pprint import pprint as pp

# String Manipulation
import re 

# Configurations
import warnings
warnings.filterwarnings("ignore")
warnings.simplefilter(action='ignore', category=FutureWarning)
pd.set_option('display.max_columns', None)

# Helpers
# -----------------------------------------------------------------------------
# To learn process time
def get_time(cond):
    if cond == "start":
        p = "This process started at "
    elif cond == "end":
        p = "The process completed at "
    print("") 
    print(colored(p + str(datetime.now().strftime("%H:%M:%S")), "green", "on_white", attrs=["bold",'reverse', 'blink']))
    
    
def Filter(string, substr): 
        return [str for str in string if
                any(sub in str for sub in substr)] 
    
def NOTFilter(string, substr): 
    return [str for str in string if
            any(sub not in str for sub in substr)] 

In [ ]:
# England: 9 | Italy: 11 | Spain: 12 | France: 13 | Germany: 20
def fbref_league_history(league_id = [9,11,12,13,20], first_season = 2017):
    history = []
    for i in league_id:
        comp_history_url = "https://fbref.com/en/comps/" + str(i) + "/history" 
        #print(comp_history_url)

        r=requests.get(comp_history_url)
        soup=BeautifulSoup(r.content, "html.parser")

        # Tablodan tüm sezonun bilgilerine ulaşılır.
        find_seasons = soup.find_all(class_ = "left")

        all_seasons_url = []
        for k in range(0, len(find_seasons)):
            if find_seasons[k].get('data-stat') == "season":
                temp = "https://fbref.com" + find_seasons[k].find_all("a")[0].attrs["href"]
                all_seasons_url.append(temp)

        history.append(all_seasons_url)
        time.sleep(0.1)

    # Tüm historyler tek bir arrayde
    history  = list(itertools.chain(*history))

    seasons = list(map(lambda x: str(x)+"-"+str(x+1), np.arange(1950, first_season, 1)))
    for i in seasons:
        history = NOTFilter(history, [i])
    del seasons

    # history = NOTFilter(history, ["2021-2022"])

    return history

# Test for Big 5
history = fbref_league_history(league_id = [9,11,12,13,20])

In [ ]:
def fbref_team_url_history(league_history):
    team_season_url = []
    for league_season_url in league_history:
        r=requests.get(league_season_url)
        soup=BeautifulSoup(r.content, "html.parser")
        teams = soup.find("table").find_all("a")
        teams = list(map(lambda x: "https://fbref.com" + x["href"], teams))
        teams = Filter(teams, ["/en/squads/"])
        team_season_url.append(teams)

    # Tüm historyler tek bir arrayde
    team_season_url  = list(itertools.chain(*team_season_url))
    return team_season_url

team_season_url = fbref_team_url_history(history)

In [ ]:
def fbref_player_url(team_season_url):
    player_url = []
    for turl in team_season_url:
        r=requests.get(turl)
        soup=BeautifulSoup(r.content, "html.parser")
        soup.find("div", {"id":"all_stats_standard"})
        players = soup.find("tbody").find_all("a")
        players = list(map(lambda x: x["href"], players))
        players = Filter(players, ["/en/players/"])
        players = NOTFilter(players, ["matchlogs"])
        player_url.append(list(map(lambda x: "https://fbref.com" + x, players)))
        time.sleep(0.01)
    player_url  = list(itertools.chain(*player_url))
    return player_url

player_url = fbref_player_url(team_season_url)

In [ ]:
def fbref_player_info(player_url):
    player_info = []
    for completed, i in enumerate(player_url):

        # PlayerId
        playerId = i.replace("https://fbref.com/en/players/", "").split("/")[0]

        # Request
        r=requests.get(i)
        soup=BeautifulSoup(r.content, "html.parser")

        # Meta
        meta = soup.find("div", {"id":"meta"})

        # Player Name

        tmp = soup.find("h1").find("span")

        if tmp is None: continue

        playerName = tmp.get_text()

        # Player Photos
        try:
            photo = soup.find("div", {"class":"media-item"}).find("img").attrs["src"]
        except:
            photo = np.nan

        detail = meta.find_all("p")

        # Player Full Name
        if len(Filter([detail[0].text], ["Position", "Club", "Born", "National Team", "Citizenship"])) > 0:
                playerFullName = np.nan
        else:
            playerFullName = detail[0].get_text()

        # Data Frame
        temp = pd.DataFrame({
            "FBRefId":[playerId],
            "PlayerName":[playerName],
            "PlayerFullName":[playerFullName],
            # "Nationality":[nationality],
            "Photo":[photo],
            # "Birth":[birth],
            # "Height":[height],
            # "Weight":[weight],
            # "Position":[position],
            # "Foot":[foot],
            # "FootAbility":[foot_ability],
            # "InternationalReputation":[ir],
            # "PlayerUrl":[i],
            # "Twitter":[tw],
            # "Instagram":[ins]
        })    

        temp["PlayerFullName"] = np.where(temp.PlayerFullName.isnull(), temp.PlayerName, temp.PlayerFullName)

        player_info.append(temp)

        # Print Message
        sys.stdout.write("\r{0} players have just scraped from FBRef!".format(completed+1))
        sys.stdout.flush()

        # System Sleep
        time.sleep(0.01) 

    # Write Player Info
    player_info = pd.concat(player_info)
    
    return player_info 

player_info = fbref_player_info(player_url[:5000])

5000 players have just scraped from FBRef!

In [ ]:
player_info_0 = fbref_player_info(player_url[:5000])
print('0 done')
player_info_1 = fbref_player_info(player_url[5000:10000])
print('1 done')

In [ ]:
player_info_2 = fbref_player_info(player_url[10000:15000])
print('2 done')
player_info_3 = fbref_player_info(player_url[15000:])
print('3 done')

5000 players have just scraped from FBRef!2 done
1176 players have just scraped from FBRef!3 done


In [ ]:
# df_photos = pd.concat([player_info_0, player_info_1, player_info_2, player_info_3]).to_csv('photos.csv')
df_photos = pd.concat([player_info_0, player_info_1, player_info_2, player_info_3])

In [ ]:
df_photos.head()

,FBRefId,PlayerName,PlayerFullName,Photo
0,3bb7b8b4,Ederson,Ederson Santana de Moraes,https://fbref.com/req/202005121/images/headsho...
0,bd6351cd,João Cancelo,João Pedro Cavaco Cancelo,https://fbref.com/req/202005121/images/headsho...
0,3eb22ec9,Bernardo Silva,Bernardo Mota Veiga de Carvalho e Silva,https://fbref.com/req/202005121/images/headsho...
0,31c69ef1,Rúben Dias,Rúben Dias,https://fbref.com/req/202005121/images/headsho...
0,6434f10d,Rodri,Rodrigo Hernández Cascante,https://fbref.com/req/202005121/images/headsho...


In [ ]:
df_top5 = pd.read_csv('top5.csv', index_col=False)

In [ ]:
df_top5 = df_top5[list(df_top5.columns)[1:]]

In [ ]:
photos = []

for name in list(df_top5.player):

    tmp = list(df_photos[df_photos.PlayerName == name].Photo)

    if len(tmp) == 0:
        photos.append("")
    else:
        photos.append(tmp[0])

In [ ]:
df_top5["photo"] = photos

In [ ]:
list(df_top5[df_top5.player == "Robert Lewandowski"].photo)

['https://fbref.com/req/202005121/images/headshots/8d78e732_2018.jpg']

In [ ]:
df_top5.to_csv('top5_with_photos.csv')